In [1]:
import pandas as pd
import jieba
from collections import Counter
import re
from fuzzywuzzy import fuzz
import jieba.analyse 

In [2]:
# pip install --upgrade jieba

In [3]:
filenames = [
    '广州1.csv',
    '广州2.csv',
    '广州3.csv',
    '广州4.csv',
    '广州5.csv',
    '广州6.csv'
]

dataframes = {}

for i, filename in enumerate(filenames, 1):
    print(f"Read dataset: {filename}")
    dataframes[f'guangzhou{i}'] = pd.read_csv(filename)

Read dataset: 广州1.csv
Read dataset: 广州2.csv
Read dataset: 广州3.csv
Read dataset: 广州4.csv
Read dataset: 广州5.csv
Read dataset: 广州6.csv


In [59]:
# 定义关键词字典
keywords = {
    "半导体与集成电路": [
        "晶圆制造", "晶体管", "微电子", "纳米技术", "光刻技术", "半导体材料", "IC设计", "芯片制造",
        "封装测试", "模拟电路", "数字电路", "半导体设备", "高性能计算", "存储器", "逻辑电路", 
        "FPGA", "集成电路设计", "半导体工艺", "SOC（系统级芯片）", "光子集成电路",
        "电源管理芯片", "晶体管密度", "智能传感器", "半导体产业链", "电子设计自动化（EDA）", 
        "MOSFET", "硅片", "碳化硅", "氮化镓", "微机电系统（MEMS）","半导体与集成电路","半导体","集成电路"
    ],
    "人工智能": [
        "机器学习", "深度学习", "神经网络", "自然语言处理", "计算机视觉", "强化学习", "数据挖掘",
        "大数据分析", "人工智能算法", "智能助手", "语音识别", "图像识别", "人工智能芯片", 
        "云计算", "边缘计算", "神经形态计算", "智能推荐系统", "生成对抗网络（GAN）", "深度伪造",
        "智能医疗", "智能制造", "自动化流程", "人工智能伦理", "强人工智能", "弱人工智能",
        "智能城市", "数据隐私", "虚拟助手", "智能搜索", "智能感知","人工智能"
    ],
    "低空经济与空天": [
        "无人机", "低空飞行", "航空器", "空域管理", "卫星导航", "空天技术", "低空旅游", 
        "城市空中交通", "空域安全", "机载传感器", "遥感技术", "空天通信", "地面站", 
        "空中交通管理", "空中物流", "低轨道卫星", "气象监测", "空中成像", "航空经济", 
        "空域开放", "航空监管", "低空空域", "空中预警", "飞行器制造", "空天探测", 
        "航空安全", "城市空域", "空中加油", "无人机物流", "空中交通管制","低空经济与空天"
    ],
    "新能源": [
        "太阳能", "风能", "生物质能", "氢能", "储能技术", "可再生能源", "光伏发电", 
        "风力发电", "电动汽车", "能源转换", "能源存储", "清洁能源", "能源效率", "绿色能源", 
        "智能电网", "分布式能源", "能源管理系统", "储能电池", "燃料电池", "热能发电", 
        "氢燃料电池", "核能", "潮汐能", "可持续能源", "环保技术", "能源转型", "低碳经济", 
        "微电网", "智能家居", "碳中和","新能源"
    ],
    "智能网联汽车": [
        "车联网", "电动汽车", "智能交通", "车载传感器", "人机交互", "LIDAR", 
        "车载计算", "车载操作系统", "车载娱乐系统", "车联网安全", "远程更新", "智能导航", 
        "智能交通系统", "车内智能助手", "无线充电", "智能调度", "智能停车", "车路协同", 
        "车内物联网", "电动动力系统", "自动泊车", "V2X通信", "车载AI芯片", "车辆安全系统", 
        "车内健康监测", "智能网联平台", "远程诊断", "数字仪表盘", "智能出行", "自动驾驶传感器",
        "智能网联汽车"
    ],
    "生物医药": [
        "生物技术", "基因编辑", "疫苗研发", "生物制药", "免疫治疗", "精准医疗", "合成生物学", 
        "干细胞", "临床试验", "药物研发", "基因组学", "生物标志物", "生物信息学", 
        "蛋白质组学", "分子诊断", "基因治疗", "细胞疗法", "生物仿制药", "医药大数据", 
        "生物工程", "临床基因组", "医疗器械", "生物材料", "药物递送", "再生医学", 
        "新药研发", "疫苗生产", "疾病诊断", "免疫检测", "生物传感器","生物医药"
    ],
    "高端医疗器械": [
        "医疗影像", "手术机器人", "植入医疗器械", "智能医疗设备", "体外诊断", "远程医疗", 
        "医疗传感器", "高端监护设备", "可穿戴设备", "医疗AI", "数字医疗", 
        "微创手术", "诊断设备", "精密仪器", "放射治疗", "心血管设备", "神经调控设备", 
        "康复设备", "3D打印医疗器械", "医疗成像", "药物输送系统", "电子病历", 
        "可视化手术", "临床分析", "医用激光", "医疗机器人", "基因测序", "医疗数据管理", 
        "远程监控", "电子听诊器","高端医疗器械","医疗器械"
    ],
    "网络与通信": [
        "5G", "光纤通信", "移动通信", "物联网", "网络安全", "云计算", "边缘计算", "数据中心", 
        "网络架构", "移动宽带", "卫星通信", "无线网络", "网络协议", "软件定义网络", 
        "网络虚拟化", "通信设备", "移动设备", "大数据传输", "宽带网络", "下一代网络", 
        "量子通信", "网络切片", "远程通信", "智能天线", "通信标准", "网络运营","网络","通信"
        "通信基础设施", "低延迟通信", "数字通信", "数据加密", "异构网络","网络与通信"
    ],
    "超高清视频显示": [
        "4K", "8K", "高动态范围（HDR）", "有机发光二极管（OLED）", "微型LED", "液晶显示器（LCD）", 
        "屏幕分辨率", "显示面板", "显示技术", "投影显示", "色彩校准", "屏幕刷新率", "图像处理", 
        "高帧率（HFR）", "图像增强", "显示驱动芯片", "屏幕亮度", "显示器制造", "屏幕对比度", 
        "屏幕尺寸", "显示控制器", "视觉体验", "画质优化", "视觉内容创作", "视频压缩", 
        "流媒体传输", "超高清摄像机", "虚拟现实（VR）", "增强现实（AR）", "显示屏设计", "影院显示",
        "超高清视频显示"
    ],
    "智能终端": [
        "智能手表", "平板电脑", "智能家居设备", "语音助手", "可穿戴设备", "物联网终端", 
        "虚拟现实头显", "增强现实眼镜", "智能音箱", "物联网网关", "智能电视", "智能家电", 
        "智能摄像头", "车载智能终端", "智能门锁", "健康监测设备", "智能耳机", "智能照明", 
        "智能温控器", "智能手环", "智能玩具", "智能投影仪", "智能显示屏", "智能遥控器", 
        "语音识别技术", "面部识别技术", "多媒体处理", "智能传感器", "云端连接", "移动支付",
        "智能终端" 
    ],
    "软件与信息服务": [
        "软件开发", "云计算服务", "大数据分析", "人工智能软件", "企业资源规划（ERP）", 
        "客户关系管理（CRM）", "网络安全软件", "移动应用开发", "开发运维（DevOps）", 
        "软件即服务（SaaS）", "平台即服务（PaaS）", "基础设施即服务（IaaS）", "数据存储", 
        "计算机视觉", "自然语言处理", "数字化转型", "自动化测试", "企业软件", "移动安全", 
        "API管理", "云原生应用", "虚拟化技术", "数据中心管理", "软件架构", "网络管理", 
        "软件更新", "开源软件", "人工智能驱动软件", "机器学习平台", "区块链技术", "数字身份认证",
        "软件与信息服务","软件","信息服务"
    ],
    "智能传感器": [
        "MEMS传感器", "光传感器", "压力传感器", "加速度传感器", "温度传感器", "湿度传感器",
        "气体传感器", "位置传感器", "生物传感器", "环境监测传感器", "运动传感器", "超声波传感器",
        "红外传感器", "指纹识别传感器", "人脸识别传感器", "多传感器融合", "传感器网络", "智能检测",
        "智能制造", "物联网传感器", "工业自动化", "智能汽车传感器", "医疗传感器", "智能家居传感器",
        "数据采集", "能量采集", "无线传感器网络", "微型传感器", "自适应传感器", "电化学传感器","智能传感器"
    ],
    "高端装备与仪器": [
        "精密机械", "高速加工", "数控机床", "工业机器人", "增材制造", "激光切割", "工业自动化",
        "航空航天装备", "高性能材料", "轨道交通设备", "船舶制造", "医疗设备", "智能制造",
        "自动化控制", "工业物联网", "精密仪器", "测量与检测", "电力设备", "机械设计",
        "动力系统", "高端传感器", "智能工厂", "能源装备", "数字孪生", "机器人焊接",
        "智能维护", "高性能合金", "大型工程机械", "智能控制系统", "自动化物流设备","高端装备与仪器","高端装备","高端仪器"
    ],
    "机器人": [
        "工业机器人", "服务机器人", "协作机器人", "医疗机器人", "机器人感知",
        "机器人导航", "机器人控制", "人工智能机器人", "机器人学", "自主移动机器人",
        "机器人操作系统（ROS）", "机器人视觉", "机器人听觉", "机器人手臂", "机器人编程",
        "机器人制造", "自动化机器人", "机器人集成", "机器人操作平台", "机器人传感器",
        "机器人测试", "仿生机器人", "水下机器人", "机器人抓取", "机器人焊接", "机器人装配",
        "人形机器人", "机器人学习", "智能自动化"
    ],
    "高性能材料": [
        "碳纤维", "复合材料", "陶瓷材料", "高强度钢", "钛合金", "石墨烯", "纳米材料", "高温合金",
        "超导材料", "形状记忆合金", "功能材料", "新型涂层", "超硬材料", "超轻材料", "阻燃材料",
        "生物材料", "难熔金属", "高性能塑料", "材料加工技术", "新型金属材料", "材料表面处理",
        "智能材料", "环保材料", "材料测试", "纳米复合材料", "热电材料", "自愈材料", "高导电材料",
        "绝缘材料", "储能材料", "高性能材料"
    ],
    "数字创意": [
        "数字媒体", "数字艺术", "游戏开发", "虚拟现实", "增强现实", "数字内容创作", "数字动画",
        "计算机图形学", "数字设计", "交互设计", "用户体验", "数字广告", "数字音频", "数字视频",
        "数字出版", "数字游戏", "数字影视制作", "数字广告投放", "数字营销", "数字内容分发",
        "数字展示", "数字娱乐", "数字策展", "数字艺术馆", "数字影像处理", "数字摄影", "数字雕塑",
        "数字角色设计", "数字建筑设计", "数字动画制作","数字创意","数字传媒","数字传播",'舞台灯光','音响设备',
        '视频直播','数字广告','广告','文艺创作','文化传媒','文化传播','策划创意'
    ],
    "现代时尚": [
        "智能服装", "可穿戴设备", "环保时尚", "数字时尚", "时尚设计", "快时尚", "可持续时尚",
        "时尚科技", "3D打印时尚", "数字化服装设计", "时尚零售", "虚拟试衣", "时尚电商",
        "智能穿戴", "时尚品牌", "时尚营销", "时尚趋势预测", "环保材料", "时尚奢侈品",
        "服装制造", "数字化供应链", "时尚大数据", "时尚新媒体", "时尚创意", "时尚产业链",
        "纺织科技", "时尚展览", "时尚设计师", "数字时尚营销", "时尚消费行为","现代时尚","珠宝"
    ],
    "安全节能环保": [
        "节能技术", "清洁能源", "环保设备", "环保材料", "废物处理", "环保工程", "水处理技术",
        "空气净化", "能源管理", "绿色建筑", "环境监测", "环保政策", "碳排放", "碳中和",
        "可再生能源", "生态修复", "节能减排", "资源回收", "清洁生产", "污染防治", "环保科技",
        "环保认证", "环境保护", "节能环保产业", "智能环保", "环境友好产品", "绿色供应链",
        "节能环保设备", "能效管理", "可持续发展","安全节能环保"
    ],
    "大健康": [
        "健康管理", "医疗保健", "健康大数据", "远程医疗", "基因检测", "疾病预防", "慢病管理",
        "营养保健", "健康保险", "个人健康档案", "康复治疗", "心理健康", "运动健康", "中医养生",
        "健康产业", "健康饮食", "智能健康设备", "健康咨询", "健康生活方式", "健康教育",
        "疫苗接种", "健康评估", "疾病监测", "健康服务", "医疗大数据", "健康社区", "健康技术",
        "健康城市", "健康科技", "健康养老","大健康"
    ],
    "海洋经济": [
        "海洋资源", "海洋渔业", "海洋能源", "海洋运输", "海洋生态", "海洋环境保护",
        "海洋科技", "海洋工程", "海洋旅游", "海洋矿产", "海洋装备", "海洋生物技术",
        "海洋信息服务", "海洋可持续发展", "海洋监测", "海洋开发", "海洋污染防治",
        "海洋风电", "深海探测", "海洋生物资源", "海洋牧场", "海洋食品加工", "海洋空间规划",
        "海洋保护区", "海洋经济区", "海洋灾害预警", "海水淡化", "海洋学研究", "海上风能",
        "海洋安全","海洋经济"
    ],
    "合成生物": [
        "基因编辑", "DNA合成", "合成酶", "基因线路", "合成生命", "代谢工程", "蛋白质设计", 
        "生物元件", "细胞工厂", "微生物工程", "生物安全", "生物计算", "生物传感器", 
        "生物电池", "基因组优化", "生物合成", "合成生物材料", "生物燃料", "生物医药", 
        "基因组学", "生物芯片", "微流控技术", "合成抗体", "生物制造", "基因治疗", 
        "代谢产物", "细胞模拟器", "合成微生物", "基因回路设计", "生物催化剂","合成生物"
    ],
    "光载信息": [
        "光通信", "光子学", "光纤网络", "光调制器", "光子集成", "光量子计算", 
        "光传感器", "光交换", "光网络优化", "光波分复用", "光纤激光器", "光电子技术", 
        "光纤传感", "光纤传输", "光通信芯片", "光载微波", "光子晶体", "光学放大器", 
        "光网络单元", "光波导", "光耦合器", "光子传输", "光纤布线", "光纤连接", 
        "光传输系统", "光传输设备", "光纤通信技术", "光纤网络拓扑", "光纤传输质量", 
        "光纤通信系统","光载信息"
    ],
    "智能机器人": [
        "自主导航", "人机交互", "机器人感知", "机器人手臂", "智能制造", "工业机器人", 
        "机器视觉", "服务机器人", "医疗机器人", "机器人仿真", "人形机器人", 
        "协作机器人", "机器人算法", "机器臂末端执行器", "智能交通", "多机器人系统", 
        "机器人抓取", "机器人操作系统", "机器人编程", "强化学习", "机器人感应器", 
        "机器人控制器", "智能机器人平台", "机器人学习", "仿生机器人", "智能抓取", 
        "机器人安全", "无人值守", "工业自动化机器人", "机器人路径规划","智能机器人"
    ],
    "细胞与基因": [
        "干细胞", "细胞培养", "细胞分化", "免疫疗法", "CRISPR技术", "癌症基因", 
        "基因检测", "细胞再生", "遗传疾病", "细胞信号传导", "转基因技术", "基因工程", 
        "单细胞分析", "细胞遗传学", "基因组修饰", "细胞周期", "细胞凋亡", "基因表达", 
        "细胞疗法", "细胞分裂", "细胞增殖", "基因组编辑", "细胞标记", "细胞系", 
        "干细胞治疗", "细胞分离", "细胞信号通路", "基因序列", "细胞免疫", "细胞骨架",
        "细胞与基因","细胞","基因"
    ],
    "脑科学与脑机工程": [
        "神经网络", "脑电图", "脑机接口", "神经科学", "脑部成像", "神经调控", 
        "神经元", "脑功能测绘", "脑信号处理", "认知神经科学", "人工神经网络", 
        "脑部刺激", "神经工程", "神经传导", "脑功能探测", "脑部疾病", "脑电生理", 
        "神经修复", "神经系统疾病", "神经调节", "神经传感器", "脑皮层研究", 
        "脑信息处理", "脑结构分析", "神经重塑", "神经刺激", "脑-计算机接口", 
        "神经网络模型", "脑功能研究", "脑影像处理","脑科学与脑机工程","脑科学"
    ],
    "深地深海": [
        "海洋探测", "海底通信", "海洋工程", "海底矿产", "海底能源", "深海技术", 
        "海洋生物", "海洋生态", "深海探测器", "海底光缆", "海洋气象", "深海开采", 
        "海洋资源", "海底地震", "海洋地质", "海洋光学", "深海科学", "海洋生物多样性", 
        "海洋污染", "海底测绘", "深海遥感", "海底勘探", "海洋声学", "海洋观测", 
        "海底沉积物", "深海技术装备", "海底高压", "深海矿物", "海洋能量", "深海开发",
        "深地深海"
    ],
    "量子信息": [
        "量子通信", "量子计算", "量子加密", "量子纠缠", "量子测量", "量子传感", 
        "量子算法", "量子比特", "量子网络", "量子芯片", "量子态", "量子模拟", 
        "量子纠缠态", "量子密钥分发", "量子隐形传态", "量子信息处理", "量子系统", 
        "量子随机数", "量子密码学", "量子优化", "量子人工智能", "量子计算机", 
        "量子多体问题", "量子光学", "量子克隆", "量子信息科学", "量子态转移", 
        "量子态控制", "量子隐形传送", "量子噪声","量子信息"
    ],
    "前沿新材料": [
        "石墨烯", "纳米材料", "超导材料", "复合材料", "碳纤维", "新能源材料", 
        "高温合金", "功能材料", "智能材料", "生物材料", "新型陶瓷", "记忆合金", 
        "自修复材料", "超轻材料", "柔性电子", "增材制造", "透明导电膜", "磁性材料", 
        "电池材料", "氧化物材料", "超高强钢", "纳米结构", "纳米涂层", "生物降解材料", 
        "有机光电材料", "储氢材料", "量子点材料", "超硬材料", "光电材料", "低维材料",
        "前沿新材料"
    ]
}

## jieba

In [35]:
# 将关键词加入词典里
words = [
    "晶圆制造", "晶体管", "微电子", "纳米技术", "光刻技术", "半导体材料", "IC设计", "芯片制造",
    "封装测试", "模拟电路", "数字电路", "半导体设备", "高性能计算", "存储器", "逻辑电路", 
    "FPGA", "集成电路设计", "半导体工艺", "SOC（系统级芯片）", "光子集成电路",
    "电源管理芯片", "晶体管密度", "智能传感器", "半导体产业链", "电子设计自动化（EDA）", 
    "MOSFET", "硅片", "碳化硅", "氮化镓", "微机电系统（MEMS）",
    "机器学习", "深度学习", "神经网络", "自然语言处理", "计算机视觉", "强化学习", "数据挖掘",
    "大数据分析", "人工智能算法", "智能助手", "语音识别", "图像识别", "人工智能芯片", 
    "云计算", "边缘计算", "神经形态计算", "智能推荐系统", "生成对抗网络（GAN）", "深度伪造",
    "智能医疗", "智能制造", "自动化流程", "人工智能伦理", "强人工智能", "弱人工智能",
    "智能城市", "数据隐私", "虚拟助手", "智能搜索", "智能感知",
    "无人机", "低空飞行", "航空器", "空域管理", "卫星导航", "空天技术", "低空旅游", 
    "城市空中交通", "空域安全", "机载传感器", "遥感技术", "空天通信", "地面站", 
    "空中交通管理", "空中物流", "低轨道卫星", "气象监测", "空中成像", "航空经济", 
    "空域开放", "航空监管", "低空空域", "空中预警", "飞行器制造", "空天探测", 
    "航空安全", "城市空域", "空中加油", "无人机物流", "空中交通管制",
    "太阳能", "风能", "生物质能", "氢能", "储能技术", "可再生能源", "光伏发电", 
    "风力发电", "电动汽车", "能源转换", "能源存储", "清洁能源", "能源效率", "绿色能源", 
    "智能电网", "分布式能源", "能源管理系统", "储能电池", "燃料电池", "热能发电", 
    "氢燃料电池", "核能", "潮汐能", "可持续能源", "环保技术", "能源转型", "低碳经济", 
    "微电网", "智能家居", "碳中和",
    "车联网", "电动汽车", "智能交通", "车载传感器", "人机交互", "LIDAR", 
    "车载计算", "车载操作系统", "车载娱乐系统", "车联网安全", "远程更新", "智能导航", 
    "智能交通系统", "车内智能助手", "无线充电", "智能调度", "智能停车", "车路协同", 
    "车内物联网", "电动动力系统", "自动泊车", "V2X通信", "车载AI芯片", "车辆安全系统", 
    "车内健康监测", "智能网联平台", "远程诊断", "数字仪表盘", "智能出行", "自动驾驶传感器",
    "生物技术", "基因编辑", "疫苗研发", "生物制药", "免疫治疗", "精准医疗", "合成生物学", 
    "干细胞", "临床试验", "药物研发", "基因组学", "生物标志物", "生物信息学", 
    "蛋白质组学", "分子诊断", "基因治疗", "细胞疗法", "生物仿制药", "医药大数据", 
    "生物工程", "临床基因组", "医疗器械", "生物材料", "药物递送", "再生医学", 
    "新药研发", "疫苗生产", "疾病诊断", "免疫检测", "生物传感器",
    "医疗影像", "手术机器人", "植入医疗器械", "智能医疗设备", "体外诊断", "远程医疗", 
    "医疗传感器", "高端监护设备", "可穿戴设备", "医疗AI", "数字医疗", 
    "微创手术", "诊断设备", "精密仪器", "放射治疗", "心血管设备", "神经调控设备", 
    "康复设备", "3D打印医疗器械", "医疗成像", "药物输送系统", "电子病历", 
    "可视化手术", "临床分析", "医用激光", "医疗机器人", "基因测序", "医疗数据管理", 
    "远程监控", "电子听诊器",
    "5G", "光纤通信", "移动通信", "物联网", "网络安全", "云计算", "边缘计算", "数据中心", 
    "网络架构", "移动宽带", "卫星通信", "无线网络", "网络协议", "软件定义网络", 
    "网络虚拟化", "通信设备", "移动设备", "大数据传输", "宽带网络", "下一代网络", 
    "量子通信", "网络切片", "远程通信", "智能天线", "通信标准", "网络运营", 
    "通信基础设施", "低延迟通信", "数字通信", "数据加密", "异构网络",
    "4K", "8K", "高动态范围（HDR）", "有机发光二极管（OLED）", "微型LED", "液晶显示器（LCD）", 
    "屏幕分辨率", "显示面板", "显示技术", "投影显示", "色彩校准", "屏幕刷新率", "图像处理", 
    "高帧率（HFR）", "图像增强", "显示驱动芯片", "屏幕亮度", "显示器制造", "屏幕对比度", 
    "屏幕尺寸", "显示控制器", "视觉体验", "画质优化", "视觉内容创作", "视频压缩", 
    "流媒体传输", "超高清摄像机", "虚拟现实（VR）", "增强现实（AR）", "显示屏设计", "影院显示",
    "智能手表", "平板电脑", "智能家居设备", "语音助手", "可穿戴设备", "物联网终端", 
    "虚拟现实头显", "增强现实眼镜", "智能音箱", "物联网网关", "智能电视", "智能家电", 
    "智能摄像头", "车载智能终端", "智能门锁", "健康监测设备", "智能耳机", "智能照明", 
    "智能温控器", "智能手环", "智能玩具", "智能投影仪", "智能显示屏", "智能遥控器", 
    "语音识别技术", "面部识别技术", "多媒体处理", "智能传感器", "云端连接", "移动支付",
    "软件开发", "云计算服务", "大数据分析", "人工智能软件", "企业资源规划（ERP）", 
    "客户关系管理（CRM）", "网络安全软件", "移动应用开发", "开发运维（DevOps）", 
    "软件即服务（SaaS）", "平台即服务（PaaS）", "基础设施即服务（IaaS）", "数据存储", 
    "计算机视觉", "自然语言处理", "数字化转型", "自动化测试", "企业软件", "移动安全", 
    "API管理", "云原生应用", "虚拟化技术", "数据中心管理", "软件架构", "网络管理", 
    "软件更新", "开源软件", "人工智能驱动软件", "机器学习平台", "区块链技术", "数字身份认证",
    "MEMS传感器", "光传感器", "压力传感器", "加速度传感器", "温度传感器", "湿度传感器",
    "气体传感器", "位置传感器", "生物传感器", "环境监测传感器", "运动传感器", "超声波传感器", 
    "红外传感器", "指纹识别传感器", "人脸识别传感器", "多传感器融合", "传感器网络", "智能检测",
    "智能制造", "物联网传感器", "工业自动化", "智能汽车传感器", "医疗传感器", "智能家居传感器",
    "数据采集", "能量采集", "无线传感器网络", "微型传感器", "自适应传感器", "电化学传感器",
    "精密机械", "高速加工", "数控机床", "工业机器人", "增材制造", "激光切割", "工业自动化",
    "航空航天装备", "高性能材料", "轨道交通设备", "船舶制造", "医疗设备", "智能制造",
    "自动化控制", "工业物联网", "精密仪器", "测量与检测", "电力设备", "机械设计",
    "动力系统", "高端传感器", "智能工厂", "能源装备", "数字孪生", "机器人焊接",
    "智能维护", "高性能合金", "大型工程机械", "智能控制系统", "自动化物流设备",
    "工业机器人", "服务机器人", "协作机器人", "医疗机器人", "机器人感知",
    "机器人导航", "机器人控制", "人工智能机器人", "机器人学", "自主移动机器人",
    "机器人操作系统（ROS）", "机器人视觉", "机器人听觉", "机器人手臂", "机器人编程",
    "机器人制造", "自动化机器人", "机器人集成", "机器人操作平台", "机器人传感器",
    "机器人测试", "仿生机器人", "水下机器人", "机器人抓取", "机器人焊接", "机器人装配",
    "人形机器人", "机器人学习", "智能自动化",
    "碳纤维", "复合材料", "陶瓷材料", "高强度钢", "钛合金", "石墨烯", "纳米材料", "高温合金",
    "超导材料", "形状记忆合金", "功能材料", "新型涂层", "超硬材料", "超轻材料", "阻燃材料",
    "生物材料", "难熔金属", "高性能塑料", "材料加工技术", "新型金属材料", "材料表面处理",
    "智能材料", "环保材料", "材料测试", "纳米复合材料", "热电材料", "自愈材料", "高导电材料",
    "绝缘材料", "储能材料",
    "数字媒体", "数字艺术", "游戏开发", "虚拟现实", "增强现实", "数字内容创作", "数字动画",
    "计算机图形学", "数字设计", "交互设计", "用户体验", "数字广告", "数字音频", "数字视频",
    "数字出版", "数字游戏", "数字影视制作", "数字广告投放", "数字营销", "数字内容分发",
    "数字展示", "数字娱乐", "数字策展", "数字艺术馆", "数字影像处理", "数字摄影", "数字雕塑",
    "数字角色设计", "数字建筑设计", "数字动画制作",
    "智能服装", "可穿戴设备", "环保时尚", "数字时尚", "时尚设计", "快时尚", "可持续时尚",
    "时尚科技", "3D打印时尚", "数字化服装设计", "时尚零售", "虚拟试衣", "时尚电商",
    "智能穿戴", "时尚品牌", "时尚营销", "时尚趋势预测", "环保材料", "时尚奢侈品",
    "服装制造", "数字化供应链", "时尚大数据", "时尚新媒体", "时尚创意", "时尚产业链",
    "纺织科技", "时尚展览", "时尚设计师", "数字时尚营销", "时尚消费行为",
    "节能技术", "清洁能源", "环保设备", "环保材料", "废物处理", "环保工程", "水处理技术",
    "空气净化", "能源管理", "绿色建筑", "环境监测", "环保政策", "碳排放", "碳中和",
    "可再生能源", "生态修复", "节能减排", "资源回收", "清洁生产", "污染防治", "环保科技",
    "环保认证", "环境保护", "节能环保产业", "智能环保", "环境友好产品", "绿色供应链",
    "节能环保设备", "能效管理", "可持续发展",
    "健康管理", "医疗保健", "健康大数据", "远程医疗", "基因检测", "疾病预防", "慢病管理",
    "营养保健", "健康保险", "个人健康档案", "康复治疗", "心理健康", "运动健康", "中医养生",
    "健康产业", "健康饮食", "智能健康设备", "健康咨询", "健康生活方式", "健康教育",
    "疫苗接种", "健康评估", "疾病监测", "健康服务", "医疗大数据", "健康社区", "健康技术",
    "健康城市", "健康科技", "健康养老",
    "海洋资源", "海洋渔业", "海洋能源", "海洋运输", "海洋生态", "海洋环境保护",
    "海洋科技", "海洋工程", "海洋旅游", "海洋矿产", "海洋装备", "海洋生物技术",
    "海洋信息服务", "海洋可持续发展", "海洋监测", "海洋开发", "海洋污染防治",
    "海洋风电", "深海探测", "海洋生物资源", "海洋牧场", "海洋食品加工", "海洋空间规划",
    "海洋保护区", "海洋经济区", "海洋灾害预警", "海水淡化", "海洋学研究", "海上风能",
    "海洋安全",
    "基因编辑", "DNA合成", "合成酶", "基因线路", "合成生命", "代谢工程", "蛋白质设计", 
    "生物元件", "细胞工厂", "微生物工程", "生物安全", "生物计算", "生物传感器", 
    "生物电池", "基因组优化", "生物合成", "合成生物材料", "生物燃料", "生物医药", 
    "基因组学", "生物芯片", "微流控技术", "合成抗体", "生物制造", "基因治疗", 
    "代谢产物", "细胞模拟器", "合成微生物", "基因回路设计", "生物催化剂",
    "光通信", "光子学", "光纤网络", "光调制器", "光子集成", "光量子计算", 
    "光传感器", "光交换", "光网络优化", "光波分复用", "光纤激光器", "光电子技术", 
    "光纤传感", "光纤传输", "光通信芯片", "光载微波", "光子晶体", "光学放大器", 
    "光网络单元", "光波导", "光耦合器", "光子传输", "光纤布线", "光纤连接", 
    "光传输系统", "光传输设备", "光纤通信技术", "光纤网络拓扑", "光纤传输质量", 
    "光纤通信系统",
    "自主导航", "人机交互", "机器人感知", "机器人手臂", "智能制造", "工业机器人", 
    "机器视觉", "服务机器人", "医疗机器人", "机器人仿真", "人形机器人", 
    "协作机器人", "机器人算法", "机器臂末端执行器", "智能交通", "多机器人系统", 
    "机器人抓取", "机器人操作系统", "机器人编程", "强化学习", "机器人感应器", 
    "机器人控制器", "智能机器人平台", "机器人学习", "仿生机器人", "智能抓取", 
    "机器人安全", "无人值守", "工业自动化机器人", "机器人路径规划",
    "干细胞", "细胞培养", "细胞分化", "免疫疗法", "CRISPR技术", "癌症基因", 
    "基因检测", "细胞再生", "遗传疾病", "细胞信号传导", "转基因技术", "基因工程", 
    "单细胞分析", "细胞遗传学", "基因组修饰", "细胞周期", "细胞凋亡", "基因表达", "细胞疗法", "细胞分裂", 
    "细胞增殖", "基因组编辑", "细胞标记", "细胞系", "干细胞治疗", "细胞分离", 
    "细胞信号通路", "基因序列", "细胞免疫", "细胞骨架",
    "神经网络", "脑电图", "脑机接口", "神经科学", "脑部成像", "神经调控", 
    "神经元", "脑功能测绘", "脑信号处理", "认知神经科学", "人工神经网络", 
    "脑部刺激", "神经工程", "神经传导", "脑功能探测", "脑部疾病", "脑电生理", 
    "神经修复", "神经系统疾病", "神经调节", "神经传感器", "脑皮层研究", 
    "脑信息处理", "脑结构分析", "神经重塑", "神经刺激", "脑-计算机接口", 
    "神经网络模型", "脑功能研究", "脑影像处理",
    "海洋探测", "海底通信", "海洋工程", "海底矿产", "海底能源", "深海技术", 
    "海洋生物", "海洋生态", "深海探测器", "海底光缆", "海洋气象", "深海开采", 
    "海洋资源", "海底地震", "海洋地质", "海洋光学", "深海科学", "海洋生物多样性", 
    "海洋污染", "海底测绘", "深海遥感", "海底勘探", "海洋声学", "海洋观测", 
    "海底沉积物", "深海技术装备", "海底高压", "深海矿物", "海洋能量", "深海开发",
    "量子通信", "量子计算", "量子加密", "量子纠缠", "量子测量", "量子传感", 
    "量子算法", "量子比特", "量子网络", "量子芯片", "量子态", "量子模拟", 
    "量子纠缠态", "量子密钥分发", "量子隐形传态", "量子信息处理", "量子系统", 
    "量子随机数", "量子密码学", "量子优化", "量子人工智能", "量子计算机", 
    "量子多体问题", "量子光学", "量子克隆", "量子信息科学", "量子态转移", 
    "量子态控制", "量子隐形传送", "量子噪声",
    "石墨烯", "纳米材料", "超导材料", "复合材料", "碳纤维", "新能源材料", 
    "高温合金", "功能材料", "智能材料", "生物材料", "新型陶瓷", "记忆合金", 
    "自修复材料", "超轻材料", "柔性电子", "增材制造", "透明导电膜", "磁性材料", 
    "电池材料", "氧化物材料", "超高强钢", "纳米结构", "纳米涂层", "生物降解材料", 
    "有机光电材料", "储氢材料", "量子点材料", "超硬材料", "光电材料", "低维材料",
    "前沿新材料","量子信息","深地深海","脑科学与脑机工程","脑科学","脑机工程","细胞与基因",
    "细胞","基因","智能机器人","光载信息","合成生物","海洋经济","大健康","安全节能环保",
    "现代时尚","珠宝","数字创意","数字传媒","数字传播", "高性能材料","高端装备与仪器","高端装备",
    "高端仪器","智能传感器","信息服务","智能终端" ,"超高清视频显示","网络与通信","医疗器械","生物医药",
    "智能网联汽车","新能源","低空经济与空天","人工智能","半导体与集成电路","半导体","集成电路"

]

for word in words:
    jieba.add_word(word)

## 方法1（关键词直接匹配）

In [6]:
# # 预处理文本函数
# def preprocess_text(text):
#     tokens = jieba.lcut(text)
#     tokens = [token for token in tokens if re.match(r'[\u4e00-\u9fff]', token)]  # 只保留中文字符
#     return tokens

# # 提取每个公司经营范围中的高频词
# def extract_high_freq_words(tokens, threshold=2):
#     token_counter = Counter(tokens)
#     high_freq_words = [word for word, count in token_counter.items() if count >= threshold]
#     return high_freq_words

# # 匹配关键词的函数
# def match_keywords(description, keywords):
#     for industry, kw_list in keywords.items():
#         for keyword in kw_list:
#             if re.search(keyword, description):
#                 return industry
#     return None

# # 处理数据集的函数
# def process_dataset(file_number):
#     file_path = f'guangzhou{file_number}.csv'
#     guangzhou = pd.read_csv(file_path)
    
#     # 将经营范围文本和企业名称进行预处理
#     guangzhou['分词'] = guangzhou.apply(lambda row: preprocess_text(str(row['企业名称'])) +
#                                         preprocess_text(str(row['经营范围'])), axis=1)
    
#     # 提取高频词
#     guangzhou['高频词'] = guangzhou['分词'].apply(lambda x: extract_high_freq_words(x, threshold=2))
    
#     # 在数据集中匹配关键词
#     guangzhou['行业分类'] = guangzhou['高频词'].apply(lambda x: match_keywords(str(x), keywords))
    
#     # 筛选出匹配的公司
#     matched_companies = guangzhou.dropna(subset=['行业分类'])
#     output_file_path = f'matched_companies{file_number}.csv'
#     matched_companies.to_csv(output_file_path, index=False)
#     print(f'Processed guangzhou{file_number}.csv and saved to {output_file_path}')

# # 处理 guangzhou1 到 guangzhou6 数据集
# for i in range(1, 7):
#     process_dataset(i)

## 方法2（关键词加fuzzywuzzy匹配分数）不带经营范围缩减

In [66]:
# def preprocess_text(text):   
#     # 删除所有括号及其内部的内容
#     text = re.sub(r'（.*?）', '', text)  # 删除中文括号及其中的内容
#     text = re.sub(r'\(.*?\)', '', text)  # 删除英文括号及其中的内容
    
#     tokens = jieba.lcut(text)
    
#     tokens = [token for token in tokens if re.search(r'^[\u4e00-\u9fff]+$', token)]
    
#     return tokens

# 提取关键词（按个数）
# def extract_high_freq_words(tokens, top_n=5):
#     # 统计词频
#     token_counter = Counter(tokens)
    
#     # 获取前 top_n 的高频词
#     high_freq_words = [word for word, count in token_counter.most_common(top_n)]
    
#     return high_freq_words

# 提取每个公司经营范围中的高频词（使用Counter 可设置词频）
# def extract_high_freq_words(tokens, threshold=2):
#     token_counter = Counter(tokens)
#     high_freq_words = [word for word, count in token_counter.items() if count >= threshold]
#     return high_freq_words

# # 匹配逻辑函数
# def match_keywords(high_freq_words, keywords, threshold=400):
#     best_match = None
#     highest_score = 0
    
#     # 将高频词组合成一个字符串
#     high_freq_string = ' '.join(high_freq_words)

#     for industry, kw_list in keywords.items():
#         total_score = 0  # 初始化该行业的总分
        
#         for keyword in kw_list:
#             score = fuzz.WRatio(high_freq_string, keyword)
#             total_score += score  # 累加每个关键词的匹配得分
            
#         # 更新最高分和最佳匹配行业
#         if total_score > highest_score:
#             highest_score = total_score
#             best_match = industry
    
#     # 只有当最高分超过阈值时才返回最佳匹配行业
#     if highest_score >= threshold:
#         return best_match, highest_score
#     else:
#         return None, 0  # 或者其他指示无匹配的值

# # 示例描述文本
# description = "集成电路芯片设计及服务;集成电路制造;集成电路芯片及产品制造;集成电路销售;集成电路芯片及产品销售;集成电路设计;技术服务、技术开发、技术咨询、技术交流、技术转让、技术推广;半导体分立器件制造;半导体分立器件销售;电子产品销售;人工智能应用软件开发;人工智能基础软件开发;软件开发;软件销售;人工智能行业应用系统集成服务;人工智能通用应用系统;人工智能硬件销售;智能控制系统集成;智能家庭消费设备制造;智能家庭消费设备销售;货物进出口;技术进出口;进出口代理;"
# high_freq_words = preprocess_text(description)
# print(f"处理的高频词: {high_freq_words}")

# #  处理高频词
# # high_freq_words = extract_high_freq_words(description)
# # print(f"处理的高频词: {high_freq_words}")

# # 进行关键词匹配
# matched_industry, match_score = match_keywords(high_freq_words, keywords)
# print(f"最匹配的行业: {matched_industry}, 匹配得分: {match_score}")

处理的高频词: ['集成电路', '芯片', '设计', '及', '服务', '集成电路', '制造', '集成电路', '芯片', '及', '产品', '制造', '集成电路', '销售', '集成电路', '芯片', '及', '产品销售', '集成电路设计', '技术', '服务', '技术开发', '技术咨询', '技术', '交流', '技术转让', '技术推广', '半导体', '分立', '器件', '制造', '半导体', '分立', '器件', '销售', '电子产品', '销售', '人工智能', '应用软件', '开发', '人工智能', '基础', '软件开发', '软件开发', '软件', '销售', '人工智能', '行业', '应用', '系统集成', '服务', '人工智能', '通用', '应用', '系统', '人工智能', '硬件', '销售', '智能控制系统', '集成', '智能', '家庭', '消费', '设备', '制造', '智能', '家庭', '消费', '设备', '销售', '货物', '进出口', '技术', '进出口', '进出口', '代理']
最匹配的行业: 半导体与集成电路, 匹配得分: 26.757575757575758


## 方法3（jieba自带关键词提取+Fuzzywuzzy）

In [8]:
# 用jieba自带的关键词提取
# def preprocess_text(text):
#     # 根据分号（；）拆分文本，并提取前 10 个部分
#     parts = text.split('；')[:10]  # 注意这里使用的是中文分号
#     # 将提取的部分重新拼接成一个字符串
#     text = '；'.join(parts)
    
#     # 删除 "（不含...）" 或类似结构的内容
#     text = re.sub(r'（不含.*?）', '', text)  # 匹配中文括号
#     text = re.sub(r'\(不含.*?\)', '', text)  # 匹配英文括号
    
#     # 提取关键词，设置提取的关键词数量
#     tokens = jieba.analyse.extract_tags(text, topK=10, allowPOS=('n', 'vn', 'v'))
    
#     return tokens

## 方法4（限制经营范围+直接分词+Fuzzywuzzy）

In [65]:
# 预处理文本函数
# 分词
# 停用词表，包含常见的地名或无关词语
stopwords = [
    '广州', '广东', '中国', '中华人民共和国', 
    '广州市', '广东省', 
    '海珠区', '天河区', '越秀区', '白云区', '黄埔区', '荔湾区', '番禺区', 
    '花都区', '南沙区', '增城区', '从化区','海珠', '天河', '越秀', '白云', '黄埔', '荔湾', '番禺', 
    '花都', '南沙', '增城', '从化','上海','利海','国海','星海','海程','茶海','南海','机关','中共','省委',
    '委党','中国人民银行','中国共产党','中共广东省委'
]

def preprocess_text(text):
    # 根据分号（；）拆分文本，提取前 10 个部分
    parts = text.split('；')[:10]  # 中文分号
    
    # 将提取的部分重新拼接
    text = '；'.join(parts)
    
    # 删除所有括号及其内部的内容
    text = re.sub(r'（.*?）', '', text)  # 删除中文括号及其中的内容
    text = re.sub(r'\(.*?\)', '', text)  # 删除英文括号及其中的内容
    
    # 删除所有包含“街道”的短语
    text = re.sub(r'\b[\u4e00-\u9fff]+街道\b', '', text)
    
    # 分词
    tokens = jieba.lcut(text)
    
    # 过滤掉停用词和非中文字符
    tokens = [token for token in tokens if re.search(r'^[\u4e00-\u9fff]+$', token) and token not in stopwords]
    
    return tokens

# 匹配逻辑函数
def match_keywords(tokens, keywords, threshold=20):
    best_match = None
    highest_score = 0
    
    # 将高频词组合成一个字符串
    high_freq_string = ' '.join(tokens)
    
    for industry, kw_list in keywords.items():
        total_score = 0  # 初始化该行业的总分
        num_keywords = len(kw_list)  # 关键词数量
        
        for keyword in kw_list:
            score = fuzz.WRatio(high_freq_string, keyword)
            total_score += score  # 累加每个关键词的匹配得分
            
        # 计算平均分
        if num_keywords > 0:
            average_score = total_score / num_keywords
        else:
            average_score = 0
        
        # 更新最高分和最佳匹配行业
        if average_score > highest_score:
            highest_score = average_score
            best_match = industry
    
    # 只有当最高分超过阈值时才返回最佳匹配行业
    if highest_score >= threshold:
        return best_match, highest_score
    else:
        return None, 0

In [38]:
def process_dataset(dataframes, file_key):
    # 获取DataFrame
    guangzhou = dataframes[file_key]
    
    # 将经营范围文本和企业名称进行预处理
    guangzhou['分词'] = guangzhou.apply(
        lambda row: preprocess_text(str(row['经营范围'])) if pd.notnull(row['经营范围']) and row['经营范围'].strip() 
        else preprocess_text(str(row['企业名称'])),
        axis=1
    )
    
    # 在数据集中匹配关键词，并保存行业分类和匹配分数
    guangzhou[['行业分类', '匹配分数']] = guangzhou['分词'].apply(lambda x: pd.Series(match_keywords(x, keywords)))
    
    # 保存处理后的结果到 dataframes 字典中
    dataframes[file_key] = guangzhou
    
    print(f'Processed dataset {file_key} and updated in dataframes')

In [13]:
# 示例调用
# 处理 'guangzhou1'
# process_dataset(dataframes, 'guangzhou1')

Processed dataset guangzhou1 and saved to dataframes["matched_companies_guangzhou1"]


In [ ]:
# 遍历 dataframes 字典中的所有键，并依次处理每一个数据集
for file_key in dataframes.keys():
    # 调用处理函数
    process_dataset(dataframes, file_key)
    
    # 获取处理后的 DataFrame
    processed_df = dataframes[file_key]
    
    # 将处理后的 DataFrame 保存为 CSV 文件
    output_file_path = f'{file_key}.csv'
    processed_df.to_csv(output_file_path,encoding='utf-8-sig', index=False)
    
    print(f'Saved processed data for {file_key} to {output_file_path}')